# Enhanced MaxFlow Demo

This notebook demonstrates the extended max_flow functionality with:
1. FlowSummary analytics for bottleneck identification
2. Saturated edge detection
3. Sensitivity analysis for capacity changes (increases and decreases)
4. Min-cut identification


In [9]:
# Import required modules
from ngraph.lib.algorithms.max_flow import (
    calc_max_flow,
    run_sensitivity,
    saturated_edges,
)
from ngraph.lib.graph import StrictMultiDiGraph

## Sample Network Creation

First, let's create a sample network with known bottlenecks for our demonstrations.

In [10]:
def create_sample_network():
    """Create a sample network with known bottlenecks."""
    g = StrictMultiDiGraph()

    # Add nodes
    for node in ["A", "B", "C", "D", "E"]:
        g.add_node(node)

    # Add edges with varying capacities to create bottlenecks
    edges = [
        ("A", "B", {"capacity": 100.0, "flow": 0.0, "flows": {}, "cost": 1.0}),
        ("A", "C", {"capacity": 50.0, "flow": 0.0, "flows": {}, "cost": 1.0}),
        ("B", "D", {"capacity": 80.0, "flow": 0.0, "flows": {}, "cost": 1.0}),
        ("C", "D", {"capacity": 60.0, "flow": 0.0, "flows": {}, "cost": 1.0}),
        (
            "B",
            "E",
            {"capacity": 40.0, "flow": 0.0, "flows": {}, "cost": 1.0},
        ),  # Bottleneck!
        ("D", "E", {"capacity": 120.0, "flow": 0.0, "flows": {}, "cost": 1.0}),
    ]

    for u, v, attrs in edges:
        g.add_edge(u, v, **attrs)

    return g


# Create our sample network
g = create_sample_network()

# Display network structure
print(f"Network has {g.number_of_nodes()} nodes and {g.number_of_edges()} edges")
print("\nEdges with capacities:")
for u, v, k, d in g.edges(data=True, keys=True):
    print(f"  {u} -> {v} (key={k}): capacity={d['capacity']}")

Network has 5 nodes and 6 edges

Edges with capacities:
  A -> B (key=OOFmeRtuSou6vpZLVhPfmw): capacity=100.0
  A -> C (key=T4O7lzy-Sf--3No5uZn5ew): capacity=50.0
  B -> D (key=q3CVjlmFSMyxznlvGTKhdA): capacity=80.0
  B -> E (key=6M4a3ZwZTKuytHkSyrJvGA): capacity=40.0
  C -> D (key=QTTKXGdHT8ud6BnwMO-_lQ): capacity=60.0
  D -> E (key=hEkosiunQQizsUpPYQ5DKQ): capacity=120.0


## 1. Basic Max Flow

First, let's demonstrate the basic MaxFlow.

In [3]:
print("=== Basic Max Flow (Backward Compatible) ===")
g = create_sample_network()

# Traditional usage - returns scalar
max_flow = calc_max_flow(g, "A", "E")
print(f"Maximum flow from A to E: {max_flow}")

=== Basic Max Flow (Backward Compatible) ===
Maximum flow from A to E: 150.0


## 2. Flow Summary Analytics

Now let's explore the enhanced functionality with FlowSummary analytics that provide detailed insights into the flow solution.

In [4]:
print("=== Flow Summary Analytics ===")
g = create_sample_network()

# Enhanced usage - returns flow value and summary
flow_value, summary = calc_max_flow(g, "A", "E", return_summary=True)

print(f"Maximum flow: {flow_value}")
print(f"Total edges analyzed: {len(summary.edge_flow)}")
print(f"Reachable nodes from source: {sorted(summary.reachable)}")
print(f"Min-cut edges (bottlenecks): {len(summary.min_cut)}")

if summary.min_cut:
    print("\nMin-cut edges:")
    for edge in summary.min_cut:
        u, v, k = edge
        capacity = None
        for u_edge, v_edge, k_edge, d in g.edges(data=True, keys=True):
            if (u, v, k) == (u_edge, v_edge, k_edge):
                capacity = d.get("capacity", "unknown")
                break
        print(f"  {u} -> {v} (key={k}, capacity={capacity})")

print("\nFlow details:")
print(f"  - Total flow: {summary.total_flow}")
print(f"  - Edge flows: {len(summary.edge_flow)} edges have flow")
print(f"  - Residual capacities: {len(summary.residual_cap)} edges tracked")

=== Flow Summary Analytics ===
Maximum flow: 150.0
Total edges analyzed: 6
Reachable nodes from source: ['A']
Min-cut edges (bottlenecks): 2

Min-cut edges:
  A -> B (key=FvkLnMVUSbKLEvtsEhqZBw, capacity=100.0)
  A -> C (key=QD32uiIESSifIs7c6xKFzg, capacity=50.0)

Flow details:
  - Total flow: 150.0
  - Edge flows: 6 edges have flow
  - Residual capacities: 6 edges tracked


## 3. Bottleneck Detection

The `saturated_edges` helper function identifies which edges are fully utilized and represent bottlenecks in the network.

In [5]:
print("=== Bottleneck Detection ===")
g = create_sample_network()

# Find saturated (bottleneck) edges
saturated = saturated_edges(g, "A", "E")

print(f"Found {len(saturated)} saturated edges:")
for edge in saturated:
    u, v, k = edge
    print(f"  {u} -> {v} (key={k}) - fully utilized")

# Let's also show the edge capacities for context
print("\nSaturated edge details:")
for edge in saturated:
    u, v, k = edge
    edge_data = g.get_edge_data(u, v, k)
    if edge_data:
        print(f"  {u} -> {v} (key={k}): capacity={edge_data['capacity']}")

=== Bottleneck Detection ===
Found 3 saturated edges:
  A -> B (key=ZBE4tJf9QsOQ3JiplORHZQ) - fully utilized
  A -> C (key=qyjhuhkHSNaSXTiyix6zRg) - fully utilized
  B -> E (key=IJdfc4sxT7CQMSO8Tjgd_g) - fully utilized

Saturated edge details:
  A -> B (key=ZBE4tJf9QsOQ3JiplORHZQ): capacity=100.0
  A -> C (key=qyjhuhkHSNaSXTiyix6zRg): capacity=50.0
  B -> E (key=IJdfc4sxT7CQMSO8Tjgd_g): capacity=40.0


## 4. Sensitivity Analysis for Capacity Changes

The `run_sensitivity` function helps identify which capacity changes would have the highest impact on total flow. It supports both capacity increases (positive values) and decreases (negative values). When a capacity change would result in a negative capacity, the function automatically sets the capacity to zero.

In [11]:
print("=== Sensitivity Analysis for Capacity Changes ===")
g = create_sample_network()

# Analyze impact of increasing each bottleneck capacity
print("\n--- Capacity Increases ---")
sensitivity_increase = run_sensitivity(g, "A", "E", change_amount=20.0)

print("Impact of increasing each saturated edge by 20 units:")

# Sort by impact (highest first)
sorted_impacts = sorted(sensitivity_increase.items(), key=lambda x: x[1], reverse=True)

for edge, impact in sorted_impacts:
    u, v, k = edge
    print(f"  {u} -> {v} (key={k}): +{impact:.1f} flow increase")

if sorted_impacts:
    best_edge, best_impact = sorted_impacts[0]
    u, v, k = best_edge
    print(
        f"\nBest upgrade target: {u} -> {v} (key={k}) with +{best_impact:.1f} flow increase"
    )

    # Show current capacity for context
    edge_data = g.get_edge_data(u, v, k)
    if edge_data:
        current_cap = edge_data["capacity"]
        print(
            f"  Current capacity: {current_cap} -> Upgraded capacity: {current_cap + 20.0}"
        )

# Analyze impact of decreasing each bottleneck capacity
print("\n--- Capacity Decreases ---")
sensitivity_decrease = run_sensitivity(g, "A", "E", change_amount=-5.0)

print("Impact of decreasing each saturated edge by 5 units:")

# Sort by impact (most negative first)
sorted_impacts_dec = sorted(sensitivity_decrease.items(), key=lambda x: x[1])

for edge, impact in sorted_impacts_dec:
    u, v, k = edge
    print(f"  {u} -> {v} (key={k}): {impact:+.1f} flow change")

if sorted_impacts_dec:
    worst_edge, worst_impact = sorted_impacts_dec[0]
    u, v, k = worst_edge
    print(
        f"\nMost critical edge for reduction: {u} -> {v} (key={k}) with {worst_impact:+.1f} flow impact"
    )

# Demonstrate zero-capacity behavior for large decreases
print("\n--- Testing Large Decrease (Zero-Capacity Behavior) ---")
sensitivity_large = run_sensitivity(g, "A", "E", change_amount=-100.0)
print(
    f"Large decrease test: {len(sensitivity_large)} edges analyzed (capacities set to 0 when change would be negative)"
)

# Show the impact of setting each edge to zero capacity
if sensitivity_large:
    print("\nImpact of setting each edge capacity to zero:")
    sorted_zero_impacts = sorted(sensitivity_large.items(), key=lambda x: x[1])
    for edge, impact in sorted_zero_impacts:
        u, v, k = edge
        edge_data = g.get_edge_data(u, v, k)
        current_cap = edge_data["capacity"] if edge_data else "unknown"
        print(f"  {u} -> {v} (capacity {current_cap} -> 0): {impact:+.1f} flow change")

=== Sensitivity Analysis for Capacity Changes ===

--- Capacity Increases ---
Impact of increasing each saturated edge by 20 units:
  A -> B (key=0_mOkYdZTvajpoBNoYrH_w): +10.0 flow increase
  A -> C (key=EqYAP0u0RTOU_ydmbCbvsA): +10.0 flow increase
  B -> E (key=Jh6FuF36Tz-QkBx0y2BtuA): +0.0 flow increase

Best upgrade target: A -> B (key=0_mOkYdZTvajpoBNoYrH_w) with +10.0 flow increase
  Current capacity: 100.0 -> Upgraded capacity: 120.0

--- Capacity Decreases ---
Impact of decreasing each saturated edge by 5 units:
  A -> B (key=0_mOkYdZTvajpoBNoYrH_w): -5.0 flow change
  A -> C (key=EqYAP0u0RTOU_ydmbCbvsA): -5.0 flow change
  B -> E (key=Jh6FuF36Tz-QkBx0y2BtuA): +0.0 flow change

Most critical edge for reduction: A -> B (key=0_mOkYdZTvajpoBNoYrH_w) with -5.0 flow impact

--- Testing Large Decrease (Zero-Capacity Behavior) ---
Large decrease test: 3 edges analyzed (capacities set to 0 when change would be negative)

Impact of setting each edge capacity to zero:
  A -> B (capacity 

## 5. Advanced Sensitivity Analysis Scenarios

Let's explore more advanced scenarios to demonstrate the full capabilities of the enhanced sensitivity analysis.

In [12]:
print("=== Advanced Sensitivity Analysis Scenarios ===")
g = create_sample_network()

# Get baseline information
baseline_flow = calc_max_flow(g, "A", "E")
print(f"Baseline maximum flow: {baseline_flow}")

# Scenario 1: What if we could increase any edge by 10 units?
print("\n--- Scenario 1: Which single +10 capacity upgrade gives best ROI? ---")
sensitivity_10 = run_sensitivity(g, "A", "E", change_amount=10.0)
if sensitivity_10:
    best_edge = max(sensitivity_10.items(), key=lambda x: x[1])
    edge, impact = best_edge
    u, v, k = edge
    roi = impact / 10.0  # Flow increase per unit of capacity added
    print(f"Best investment: {u} -> {v} (ROI: {roi:.2f} flow per capacity unit)")

# Scenario 2: Risk analysis - which edge reduction hurts most?
print("\n--- Scenario 2: Risk Analysis - Most Critical Edge Vulnerabilities ---")
sensitivity_risk = run_sensitivity(g, "A", "E", change_amount=-2.0)
if sensitivity_risk:
    worst_edge = min(sensitivity_risk.items(), key=lambda x: x[1])
    edge, impact = worst_edge
    u, v, k = edge
    print(f"Most vulnerable edge: {u} -> {v} (losing 2 capacity = {impact:+.1f} flow)")

    # Show current capacity for context
    edge_data = g.get_edge_data(u, v, k)
    if edge_data:
        current_cap = edge_data["capacity"]
        utilization = -impact / 2.0  # How much flow per unit of lost capacity
        print(
            f"  Current capacity: {current_cap}, Utilization efficiency: {utilization:.2f}"
        )

# Scenario 3: Comparative analysis of different change magnitudes
print("\n--- Scenario 3: Sensitivity vs. Change Magnitude ---")
for change in [1.0, 5.0, 10.0, 20.0]:
    sens = run_sensitivity(g, "A", "E", change_amount=change)
    if sens:
        max_impact = max(sens.values())
        avg_impact = sum(sens.values()) / len(sens) if sens else 0
        print(
            f"  Change ±{change:2.0f}: Max impact {max_impact:+5.1f}, Avg impact {avg_impact:+5.1f}"
        )

=== Advanced Sensitivity Analysis Scenarios ===
Baseline maximum flow: 150.0

--- Scenario 1: Which single +10 capacity upgrade gives best ROI? ---
Best investment: A -> B (ROI: 1.00 flow per capacity unit)

--- Scenario 2: Risk Analysis - Most Critical Edge Vulnerabilities ---
Most vulnerable edge: A -> B (losing 2 capacity = -2.0 flow)
  Current capacity: 100.0, Utilization efficiency: 1.00

--- Scenario 3: Sensitivity vs. Change Magnitude ---
  Change ± 1: Max impact  +1.0, Avg impact  +0.7
  Change ± 5: Max impact  +5.0, Avg impact  +3.3
  Change ±10: Max impact +10.0, Avg impact  +6.7
  Change ±20: Max impact +10.0, Avg impact  +6.7


## 6. Combined Analysis

Now let's demonstrate the comprehensive analysis capabilities by using both return flags to get complete information in a single call.

In [8]:
print("=== Combined Analysis ===")
g = create_sample_network()

# Get all information in one call
flow_value, summary, flow_graph = calc_max_flow(
    g, "A", "E", return_summary=True, return_graph=True
)

print("Analysis Results:")
print(f"  - Maximum flow: {flow_value}")
print(f"  - Bottleneck edges: {len(summary.min_cut)}")
print(f"  - Flow graph has {flow_graph.number_of_edges()} edges with flow assignments")

# Verify flow conservation
total_source_outflow = sum(
    flow for (u, v, k), flow in summary.edge_flow.items() if u == "A"
)
print(f"  - Flow conservation check: {total_source_outflow} == {summary.total_flow}")

# Show detailed edge flow information
print("\nDetailed edge flows:")
for (u, v, k), flow in summary.edge_flow.items():
    if flow > 0:  # Only show edges with positive flow
        print(f"  {u} -> {v} (key={k}): {flow:.1f} units")

# Show residual capacities for bottleneck analysis
print("\nResidual capacities (remaining capacity):")
for (u, v, k), residual in summary.residual_cap.items():
    if residual <= 1e-10:  # Show saturated edges
        print(f"  {u} -> {v} (key={k}): {residual:.3f} (SATURATED)")
    elif residual < 10:  # Show nearly saturated edges
        print(f"  {u} -> {v} (key={k}): {residual:.1f}")

=== Combined Analysis ===
Analysis Results:
  - Maximum flow: 150.0
  - Bottleneck edges: 2
  - Flow graph has 6 edges with flow assignments
  - Flow conservation check: 150.0 == 150.0

Detailed edge flows:
  A -> B (key=r9zIPN4gQuKNbT8DzDH6oQ): 100.0 units
  A -> C (key=MCL5LU6PSa-6H156pTadog): 50.0 units
  B -> D (key=lkHnuBw3RfyOMJx3BOXn8Q): 60.0 units
  B -> E (key=O1QZZ3JvStWsavp1er_Mdg): 40.0 units
  C -> D (key=TWZAFWRZQKuDcjLvBCuowA): 50.0 units
  D -> E (key=JjT048d0RGubB9gSQyFDZg): 110.0 units

Residual capacities (remaining capacity):
  A -> B (key=r9zIPN4gQuKNbT8DzDH6oQ): 0.000 (SATURATED)
  A -> C (key=MCL5LU6PSa-6H156pTadog): 0.000 (SATURATED)
  B -> E (key=O1QZZ3JvStWsavp1er_Mdg): 0.000 (SATURATED)
